## Ensembling methods

### Ensembling

Base algorithms $b_1, \dots, b_N$ (some machine learning models)

Regression:

$a(x)=\frac{1}{N}\sum^N_{n=1}b_n(x)$

Classification:

$a(x)=\text{argmax}_{y\in \mathbb{Y}}\sum^N_{n=1}[b_n(x) = y] = mode(b_1(x),\dots,b_N(x))$

__Why does ensembling work?__

Independent base algorithms $b_1, b_2, b_3$

Binary classification task

Probability that some $b_i$ makes a mistake: $p$

Probability that ensemble makes a mistake: $p^3(3-2p)$

if $p \le 0.5$, then $p^3(3-2p) \le p$

## Bagging and random subspaces

We want to obtain independent base algorithms

Train each $b_i$ on some subset of training sample

Bagging: use boostrap (taking observations with replacement) to form subsets

Random subspaces: use a random subset of features for each algorithm

## Random forest

__Main idea: use ensembling to reduce variance, while bias stays the same__ (smooth border)

Base algorithm: decision trees

Bagging to make unique train subsets for trees

For each split in trees' noeds, use a random subset of features

With growing more trees, overfitting is unlikely

## Boosting

__Main idea: construct ensemble iteratively, correcting the mistakes of previous models__

Thus we reduce bias, and by averaging base algorithms we also might reduce variance

We can easily overfitting by growing more trees

### Boosting: training

Boosting model:

$a_N(x) = \sum^N_{n=1}b_n(x)$

On the step N, we train a model $b_N$ the following way:

$\sum^l_{i=1}(y_i, a_{N-1}(x_i) + b_N(x_i))\to\text{min}_{b_N(x)}$

### Boosting: MSE

Mean Squared Error loss function:

$L(y, \hat{y}) = (y-\hat{y})^2$

Training boosting with MSE:

$\sum^l_{i=1}(b_N(x_i) - (y_i - a_{N-1}(x_i)))^2 \to\text{min}_{b_N(x)}$

How to construct $b_N(x)$?

Residuals: $s_i^{(N)} = y_i-a_{N-1}(x_i)$

Fit the decision tree $b_N$ on residuals and train with MSE:

$\sum^l_{i=1}(b_N(x_i) - s_i^{(N)})^2 \to\text{min}_{b_N(x)}$

We have some output of our already constructed composition a, with index (N - 1) and we want to know where to move from this answer of composition to approximate this target value. This is why we take residuals as the target value for decision tree, and fit decision tree on them. We can train this residual tree just with the Mean Squared Error.

The interesting thing to understand here is that each of this decision tree solves a regression task. It doesn't matter if your initial task was a regression or a classification, still in boosting you construct this boosting ensemble out of regression trees.

__example__

Suppose that we are constructing a gradient boosting model on 5 observations. After constructing N trees, we end up with the following residual values for these observations: (-4, 2, 7, 0, 1). Suppose that we want to train the N+1-th tree, using MSE loss function. We are going to fit this tree on some target values. What will their sum be equal to? Provide the integer number.

6 (what's the profound meaning of this question?)

### Boosting: residuals problem

__Logistic loss function:__

$L(y, \hat{y}) = \log(1+\exp(-y\hat{y}))$

__Fitting on the residuals:__

$a_N(x) = sign \sum^{N}_{n=1}b_n(x)$ (take sign of sum of base algorithms)

$\sum^{l}_{i=1}\log(1+\exp(-(y_i - a_{N-1}(x_i))b_N(x_i))) \to min_{b_N(x)}$ ($y = (y_i - a_{N-1}(x_i))$, $\hat{y} = b_N(x_i)$)

Interpret above formula as follow:

If $y_i = a_{N-1}(x_i)$, then we don't use $x_i$ for training at all.   
_(if we know that our composition on the step n it predicts a correct answer already. It already work well, don't change it)_

If $y_i \ne a_{N-1}(x_i)$, then we need $|b_N(x_i)| > |\sum^{N-1}_{n=1}b_n(x)|$  
_this $b_N$ (step N) algorithm to be bigger than the sum of current base algorithms. The sum of base algorithms is a parts of currents composition. However, how big? Well, we cannot estimate correctly how big this should be, it is not a pretty stable way to train according to this loss function.__

__Mean squared Logrithmic Error:__

$L(y, \hat{y}) = (log(y+1) - log(\hat{y}+1))^2$

Fitting on the residuals:

$a_N(x) = \sum^{N}_{n=1}b_n(x)$

$\sum^{l}_{i=1}(\log(y_i - a_{N-1}(x_i) + 1) - \log(b_N(x_i) + 1))^2 \to min_{b_N(x)}$ ($y = (y_i - a_{N-1}(x_i))$, $\hat{y} = b_N(x_i)$)

If $y_i - a_{N-1}(x_i) \le -1$, then the first term is undefined. So we can see that our approach and fitting on residuals does not work for an arbitrary loss function. Here comes gradient boosting

## Gradient boosting

$\sum^{l}_{i=1}L(y_i, a_{N-1}(x_i) + b_N(x_i)) \to min_{b_N(x)}$

_Here we wants to have this composition output on the step n as close as possible to the correct answer. Current composition $a_{N-1}(x_i)$, and the tree at step N, $b_N(x_i)$ we want to grow_

Where to move from $a_{N-1}(x_i)$ (the previous base algorithm) to reduce the error

__A residual in a boosting model represents the difference between the target value and the model prediction__

Fit $b_N(x_i)$ on the pseudo-residuals:

$s^{(N)}_i = -\frac{\partial}{\partial z}L(y_i, z)|_{z=a_{N-1}(x_i)}$

The sign of $s^{(N)}_i$ shows where to move from $a_{N-1}$ to reduce the error, the absolute value - for how much  
_(here we have some current algorithm a with index $N-1$ and we want to move to some other algorithm. a with index $N$)_

In some way - a gradient descent in the space of algorithms  

_(this residual not only depends on $y_i$ and $a_{N-1}(x_i)$, also on the loss function)_

Now we deal with every loss function in a different way. 

__For example Pseudo-residual for MSE:__

$s^{(N)}_i = -\frac{\partial}{\partial z}L(y_i, z)|_{z=a_{N-1}(x_i)} = -\frac{\partial}{\partial z}\frac{1}{2}(y-z)^2|_{z=a_{N-1}(x_i)} = y_i - a_{N-1}(x_i)$

__Another example Asymmetric MSE:__

$L(y,z) = \frac{1}{2}([z < y](z-y)^2 + 10[z \ge y](z-y)^2)$

Pseudo-residual for Asymmetric MSE:

$s^{(N)}_i = -\frac{\partial}{\partial z}L(y_i, z)|_{z=a_{N-1}(x_i)} = ([z < y_i](y_i-z) + 10[z \ge y_i](y_i-z))|_{z=a_{N-1}(x_i)}$

__And logistic loss:__

$s^{(N)}_i = -\frac{\partial}{\partial z}L(y_i, z)|_{z=a_{N-1}(x_i)} = -\frac{\partial}{\partial z}log(1+\exp(-y_i z))|_{z=a_{N-1}(x_i)} = \frac{y_i}{1+\exp(y_i a_{N-1}(x_i))}$

- $y_i a_{N-1}(x_i) \to \infty \implies s^{(N)}_i \to 0$ (same sign, keep it)

- $y_i a_{N-1}(x_i) \to -\infty \implies s^{(N)}_i \to \pm1$ (diff sign, fix it)


__example__

Why is it a bad idea to fit the N-th tree in the boosting composition on residuals for the arbitrary loss function?

The specificities of the given loss function are not taken into account. Model training can go in a wrong direction if we use arbitrary loss functions.

### Summary

It is bad to fit on residuals for an arbitrary loss function 

Gradient boosting: fit on the pseudo-residuals

By doing so, we deal with every loss function differently

- Pseudo-residual for an observation depends on the current composition answer
- Pseudo-residual for an observation depends on the loss function
- Pseudo-residual for an observation depends on the target value